In [ ]:
from sympy import *
import numpy as np
import pandas as pd
from IPython.display import display

init_printing()

# Spoke stiffness matrix
Linearized force applied to the rim by a single spoke. The spoke is fixed at the hub end and the nipple end displaces by an amount $u_n$. Spoke orientation is described by unit vectors $\hat{n}_1$ along the axis and two mutually orthogonal unit vectors $\hat{n}_2, \hat{n}_3$.

Linearized elongation of the spoke is

$\Delta l = u_n \cdot \hat{n}_1$

Linearized rotation angle of the spoke is

$\Omega_1 = \frac{u_n\cdot \hat{n}_2}{l}$, 
$\Omega_2 = \frac{u_n\cdot \hat{n}_3}{l}$

Hooke's law gives the change in axial force, while the rotation of the spoke direction produces transverse forces:

$f_1 = \frac{EA}{l}\Delta l = \frac{EA}{l} u_n \cdot \hat{n}_1$

$f_2 = \frac{T}{l} \sin{\Omega_1} = \frac{T}{l} u_n \cdot \hat{n}_2$

$f_3 = \frac{T}{l} \sin{\Omega_2} = \frac{T}{l} u_n \cdot \hat{n}_3$

Finally, the vector force is

$\mathbf{f} = f_1 \hat{n}_1 + f_2 \hat{n}_2 + f_3 \hat{n}_3$

$\mathbf{f} = \left( \frac{EA}{l} \hat{n}_1\hat{n}_1 + \frac{T}{l} [\hat{n}_2\hat{n}_2 + \hat{n}_3\hat{n}_3] \right) \cdot u_n$

In [ ]:
# spoke angles and properties
a, b = symbols('alpha beta', real=True)
c1, c2, c3 = symbols('c_1 c_2 c_3', real=True)
T, ks, ls, ns, R = symbols('T k_s l_s n_s R', real=True, positive=True)

# spoke unit vector components (u,v,w)
n0 = Matrix([c1, c2, c3])

# stiffness matrix for a single spoke
k_f = (ks - T/ls)*n0*n0.T + T/ls*eye(3)

k_f

# Torque and spoke nipple offset
The displacement of the spoke nipple $u_n$ is related to the displacement of the rim shear center $u$ by

$u_n = u + \phi \hat{e}_3 \times b_s$

where $\hat{e}_3$ is the unit vector along the beam axis and $b_s$ is the position of the spoke nipple relative to the rim shear center.

In [ ]:
u, phi = symbols('u phi', real=true)
b1, b2 = symbols('b_1 b_2', real=true)

e3 = Matrix([0, 0, 1])
bs = Matrix([b1, b2, 0])

dTau_dphi = bs.cross(e3).T * k_f * bs.cross(e3)
df_dphi = k_f.dot(e3.cross(bs))

display(df_dphi)

k_s = zeros(4, 4)
k_s[0:3, 0:3] = k_f
k_s[0:3, 3] = df_dphi

k_s[3, 0] = df_dphi[0]
k_s[3, 1] = df_dphi[1]
k_s[3, 2] = df_dphi[2]

k_s[3, 3] = dTau_dphi

display(k_s)

# Continuum stiffness
The continuum stiffness per unit length is obtained by summing the components of the spoke stiffnes matrix in a cylindrical frame for all the spokes in a single periodic grouping and dividing by the arc length of the group

$\bar{k}_{spokes} = \frac{n_s}{2\pi R n_p} \sum_{i=1}^{n_p} k_{s, i}$

# Special cases
## radial spokes, no offset
$b_1 = b_2 = c_3 = 0$

In [ ]:
# Find net radial tension per unit length
k_cont = ns/(4*pi*R) * (k_s.subs([(c1, c1), (c3, 0), (b1, 0), (b2, 0)]) +\
    k_s.subs([(c1, -c1), (c3, 0), (b1, 0), (b2, 0)]))

simplify(k_cont)

## radial spokes, z-offset, no radial offset
$b_2 = c_3 = 0, b_1 = \pm d$

In [ ]:
d = symbols('d', real=true)
k_cont = ns/(4*pi*R) * (k_s.subs([(c1, c1), (c3, 0), (b1, d), (b2, 0)]) +
                        k_s.subs([(c1, -c1), (c3, 0), (b1, -d), (b2, 0)]))

simplify(k_cont)

## Symmetric-dished n-cross
$b_1=b_2=0, c_1=\pm c_1, c_3=\pm c_3$

In [ ]:
k_cont = ns/(8*pi*R) * (k_s.subs([(c1, c1), (c3, c3), (b1, 0), (b2, 0)]) +
                        k_s.subs([(c1, -c1), (c3, c3), (b1, 0), (b2, 0)]) +
                        k_s.subs([(c1, c1), (c3, -c3), (b1, 0), (b2, 0)]) +
                        k_s.subs([(c1, -c1), (c3, -c3), (b1, 0), (b2, 0)]))

simplify(k_cont * 2*pi*R/ns)

## Asymmetric-dished n-cross with arbitrary spoke offset
$c_1 = +c_{1l}, -c_{1r}$

$c_2= +c_{2l}, +c_{2r}$

$c_3 = \pm c_{3l}, \pm c_{3r}$

In [ ]:
c1l, c1r = symbols('c_1l c_1r', real=True)
c2l, c2r = symbols('c_2l c_2r', real=True)
c3l, c3r = symbols('c_3l c_3r', real=True)
lsl, lsr, ls = symbols('l_l l_r l_s', real=True)
d = symbols('d', real=True)

# Get the tensions in terms of the average net radial tension
T1, T2, Tb = symbols('T_1 T_2 T_b', real=True)
Tsol = solve([Eq(T1*c1l - T2*c1r), Eq(2*pi*R*Tb, ns*(T1*c2l + T2*c2r)/2)], [T1, T2])

display(Tsol)

k_cont = ns/(8*pi*R) * (k_s.subs([(c1, c1l), (c2, c2l), (c3, c3l), (T, T1), (b1, b1), (b2, b2), (ls, lsl)]) +
                        k_s.subs([(c1, -c1r), (c2, c2r), (c3, c3r), (T, T2), (b1, -b1), (b2, b2), (ls, lsr)]) +
                        k_s.subs([(c1, c1l), (c2, c2l), (c3, -c3l), (T, T1), (b1, b1), (b2, b2), (ls, lsl)]) +
                        k_s.subs([(c1, -c1r), (c2, c2r), (c3, -c3r), (T, T2), (b1, -b1), (b2, b2), (ls, lsr)]))

display(simplify(k_cont.subs([(T1, 0), (T2, 0)])))
display(simplify(k_cont.subs(Tsol).subs([(ks, 0), (lsl, ls), (lsr, ls)])))